In [4]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from ast import literal_eval
from statistics import median
import statistics as stats
import math
import pandas as pd
pd.set_option('display.max_columns', None)
from matplotlib import pyplot
import seaborn as sn
from scipy.stats import pearsonr
import numpy as np
from scipy import stats as s
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from dateutil.parser import *
import datetime
import random
random.seed(3)

In [63]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [64]:
train = pd.read_pickle(r'C:\Solar Credit Repayment\Zipped_Solar\train_featured_2.pkl')
test = pd.read_pickle(r'C:\Solar Credit Repayment\Zipped_Solar\test_featured_2.pkl')
# ss = pd.read_csv(r'C:\Solar Credit Repayment\SampleSubmission.csv')
# metadata = pd.read_csv(r'C:\Solar Credit Repayment\metadata.csv')
# feature_imp = pd.read_csv(r'C:\Solar Credit Repayment\Zipped_Solar\Boruta_Feature_Importance.csv')

In [65]:
#### DELETE EMA_TARGET_10
# del train['Unnamed: 0']
# del test['Unnamed: 0']


In [66]:
print(train.shape)
train.head(2)

(168042, 321)


,ID X Month,TransactionDates,PaymentsHistory,defaulted_times,ID,target,target_month_one,target_month_two,target_month_three,target_month_four,target_month_five,target_month_six,first_target_month_one,first_target_month_two,first_target_month_three,first_target_month_four,first_target_month_five,first_target_month_six,Mean_PaymentsHistory,Median_PaymentsHistory,MostCommon_PaymentsHistory,variance_PaymentsHistory,stdev_PaymentsHistory,percentile_linear_10,percentile_linear_25,percentile_linear_50,percentile_linear_75,percentile_linear_100,HighestPay_PaymentsHistory,LowestPay_PaymentsHistory,Sum_PaymentsHistory,count_PaymentsHistory,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,DaysOnDeposit_year_list,DaysOnDeposit_month_list,DaysOnDeposit_week_list,Term_year_list,Term_month_list,Term_week_list,DatetimeRegistrationDate,dayofweek_numRegistrationDate,weekdayRegistrationDate,quarterRegistrationDate,monthRegistrationDate,yearRegistrationDate,dayofyearRegistrationDate,dayofmonthRegistrationDate,weekRegistrationDate,is_month_startRegistrationDate,is_month_endRegistrationDate,is_quarter_startRegistrationDate,is_quarter_endRegistrationDate,is_year_startRegistrationDate,is_year_endRegistrationDate,SemesterRegistrationDate,DateRegistrationDate,is_weekendRegistrationDate,fortnightRegistrationDate,which_fortnightRegistrationDate,WeekOfyearRegistrationDate,HourofdayRegistrationDate,DatetimeUpsellDate,dayofweek_numUpsellDate,weekdayUpsellDate,quarterUpsellDate,monthUpsellDate,yearUpsellDate,dayofyearUpsellDate,dayofmonthUpsellDate,weekUpsellDate,is_month_startUpsellDate,is_month_endUpsellDate,is_quarter_startUpsellDate,is_quarter_endUpsellDate,is_year_startUpsellDate,is_year_endUpsellDate,SemesterUpsellDate,DateUpsellDate,is_weekendUpsellDate,fortnightUpsellDate,which_fortnightUpsellDate,WeekOfyearUpsellDate,HourofdayUpsellDate,DatetimeExpectedTermDate,dayofweek_numExpectedTermDate,weekdayExpectedTermDate,quarterExpectedTermDate,monthExpectedTermDate,yearExpectedTermDate,dayofyearExpectedTermDate,dayofmonthExpectedTermDate,weekExpectedTermDate,is_month_startExpectedTermDate,is_month_endExpectedTermDate,is_quarter_startExpectedTermDate,is_quarter_endExpectedTermDate,is_year_startExpectedTermDate,is_year_endExpectedTermDate,SemesterExpectedTermDate,DateExpectedTermDate,is_weekendExpectedTermDate,fortnightExpectedTermDate,which_fortnightExpectedTermDate,WeekOfyearExpectedTermDate,HourofdayExpectedTermDate,DatetimeFirstPaymentDate,dayofweek_numFirstPaymentDate,weekdayFirstPaymentDate,quarterFirstPaymentDate,monthFirstPaymentDate,yearFirstPaymentDate,dayofyearFirstPaymentDate,dayofmonthFirstPaymentDate,weekFirstPaymentDate,is_month_startFirstPaymentDate,is_month_endFirstPaymentDate,is_quarter_startFirstPaymentDate,is_quarter_endFirstPaymentDate,is_year_startFirstPaymentDate,is_year_endFirstPaymentDate,SemesterFirstPaymentDate,DateFirstPaymentDate,is_weekendFirstPaymentDate,fortnightFirstPaymentDate,which_fortnightFirstPaymentDate,WeekOfyearFirstPaymentDate,HourofdayFirstPaymentDate,DatetimeLastPaymentDate,dayofweek_numLastPaymentDate,weekdayLastPaymentDate,quarterLastPaymentDate,monthLastPaymentDate,yearLastPaymentDate,dayofyearLastPaymentDate,dayofmonthLastPaymentDate,weekLastPaymentDate,is_month_startLastPaymentDate,is_month_endLastPaymentDate,is_quarter_startLastPaymentDate,is_quarter_endLastPaymentDate,is_year_startLastPaymentDate,is_year_endLastPaymentDate,SemesterLastPaymentDate,DateLastPaymentDate,is_weekendLastPaymentDate,fortnightLastPaymentDate,which_fortnightLastPaymentDate,WeekOfyearLastPaymentDate,HourofdayLastPaymentDate,Age_Bin,Deposit_Bin,RatePerUnit_Bin,TotalContractValue_Bin,target_payment,first_target_payment,date_target_payment,date_first_target_payment,RatePerUnit_cat,shift_1_target_payment,shift_2_target_payment,shift_3_target_payment,sh

In [67]:
print(train.shape)
test.head(2)

(168042, 321)


,ID X Month,TransactionDates,PaymentsHistory,ID,defaulted_times,target_month_one,target_month_two,target_month_three,target_month_four,target_month_five,target_month_six,first_target_month_one,first_target_month_two,first_target_month_three,first_target_month_four,first_target_month_five,first_target_month_six,Mean_PaymentsHistory,Median_PaymentsHistory,MostCommon_PaymentsHistory,variance_PaymentsHistory,stdev_PaymentsHistory,percentile_linear_10,percentile_linear_25,percentile_linear_50,percentile_linear_75,percentile_linear_100,HighestPay_PaymentsHistory,LowestPay_PaymentsHistory,Sum_PaymentsHistory,count_PaymentsHistory,RegistrationDate,Deposit,UpsellDate,AccessoryRate,PaymentMethod,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,Town,Occupation,SupplierName,Term,TotalContractValue,ExpectedTermDate,FirstPaymentDate,LastPaymentDate,DaysOnDeposit_year_list,DaysOnDeposit_month_list,DaysOnDeposit_week_list,Term_year_list,Term_month_list,Term_week_list,DatetimeRegistrationDate,dayofweek_numRegistrationDate,weekdayRegistrationDate,quarterRegistrationDate,monthRegistrationDate,yearRegistrationDate,dayofyearRegistrationDate,dayofmonthRegistrationDate,weekRegistrationDate,is_month_startRegistrationDate,is_month_endRegistrationDate,is_quarter_startRegistrationDate,is_quarter_endRegistrationDate,is_year_startRegistrationDate,is_year_endRegistrationDate,SemesterRegistrationDate,DateRegistrationDate,is_weekendRegistrationDate,fortnightRegistrationDate,which_fortnightRegistrationDate,WeekOfyearRegistrationDate,HourofdayRegistrationDate,DatetimeUpsellDate,dayofweek_numUpsellDate,weekdayUpsellDate,quarterUpsellDate,monthUpsellDate,yearUpsellDate,dayofyearUpsellDate,dayofmonthUpsellDate,weekUpsellDate,is_month_startUpsellDate,is_month_endUpsellDate,is_quarter_startUpsellDate,is_quarter_endUpsellDate,is_year_startUpsellDate,is_year_endUpsellDate,SemesterUpsellDate,DateUpsellDate,is_weekendUpsellDate,fortnightUpsellDate,which_fortnightUpsellDate,WeekOfyearUpsellDate,HourofdayUpsellDate,DatetimeExpectedTermDate,dayofweek_numExpectedTermDate,weekdayExpectedTermDate,quarterExpectedTermDate,monthExpectedTermDate,yearExpectedTermDate,dayofyearExpectedTermDate,dayofmonthExpectedTermDate,weekExpectedTermDate,is_month_startExpectedTermDate,is_month_endExpectedTermDate,is_quarter_startExpectedTermDate,is_quarter_endExpectedTermDate,is_year_startExpectedTermDate,is_year_endExpectedTermDate,SemesterExpectedTermDate,DateExpectedTermDate,is_weekendExpectedTermDate,fortnightExpectedTermDate,which_fortnightExpectedTermDate,WeekOfyearExpectedTermDate,HourofdayExpectedTermDate,DatetimeFirstPaymentDate,dayofweek_numFirstPaymentDate,weekdayFirstPaymentDate,quarterFirstPaymentDate,monthFirstPaymentDate,yearFirstPaymentDate,dayofyearFirstPaymentDate,dayofmonthFirstPaymentDate,weekFirstPaymentDate,is_month_startFirstPaymentDate,is_month_endFirstPaymentDate,is_quarter_startFirstPaymentDate,is_quarter_endFirstPaymentDate,is_year_startFirstPaymentDate,is_year_endFirstPaymentDate,SemesterFirstPaymentDate,DateFirstPaymentDate,is_weekendFirstPaymentDate,fortnightFirstPaymentDate,which_fortnightFirstPaymentDate,WeekOfyearFirstPaymentDate,HourofdayFirstPaymentDate,DatetimeLastPaymentDate,dayofweek_numLastPaymentDate,weekdayLastPaymentDate,quarterLastPaymentDate,monthLastPaymentDate,yearLastPaymentDate,dayofyearLastPaymentDate,dayofmonthLastPaymentDate,weekLastPaymentDate,is_month_startLastPaymentDate,is_month_endLastPaymentDate,is_quarter_startLastPaymentDate,is_quarter_endLastPaymentDate,is_year_startLastPaymentDate,is_year_endLastPaymentDate,SemesterLastPaymentDate,DateLastPaymentDate,is_weekendLastPaymentDate,fortnightLastPaymentDate,which_fortnightLastPaymentDate,WeekOfyearLastPaymentDate,HourofdayLastPaymentDate,Age_Bin,Deposit_Bin,RatePerUnit_Bin,TotalContractValue_Bin,target_payment,first_target_payment,date_target_payment,date_first_target_payment,RatePerUnit_cat,shift_1_target_payment,shift_2_target_payment,shift_3_target_payment,shift_4_t

In [68]:
not_Cat_Cols = [col for col in train.columns if train[col].dtype != 'O' and train[col].dtype != 'str' and train[col].dtype != '<M8[ns]' ]
print(not_Cat_Cols)

['target', 'target_month_one', 'target_month_two', 'target_month_three', 'target_month_four', 'target_month_five', 'target_month_six', 'first_target_month_one', 'first_target_month_two', 'first_target_month_three', 'first_target_month_four', 'first_target_month_five', 'first_target_month_six', 'Mean_PaymentsHistory', 'Median_PaymentsHistory', 'MostCommon_PaymentsHistory', 'variance_PaymentsHistory', 'stdev_PaymentsHistory', 'percentile_linear_10', 'percentile_linear_25', 'percentile_linear_50', 'percentile_linear_75', 'percentile_linear_100', 'HighestPay_PaymentsHistory', 'LowestPay_PaymentsHistory', 'Sum_PaymentsHistory', 'count_PaymentsHistory', 'Deposit', 'AccessoryRate', 'RatePerUnit', 'DaysOnDeposit', 'Age', 'Term', 'TotalContractValue', 'DaysOnDeposit_year_list', 'DaysOnDeposit_month_list', 'DaysOnDeposit_week_list', 'Term_year_list', 'Term_month_list', 'Term_week_list', 'dayofweek_numRegistrationDate', 'quarterRegistrationDate', 'monthRegistrationDate', 'yearRegistrationDate', '

# Correlation with Target Variable

In [69]:
pred_dict = {}
for column in not_Cat_Cols:
    corr, _ = pearsonr(train['target'], train[column])
    pred_dict[column] = '%.3f'% corr
# Convert Dict to Dataframe
corr_df = pd.DataFrame(list(pred_dict.items()))
corr_df.sort_values(by=[1], inplace=True, ascending=False)

# Aggregation

### RATE PER UNIT AND ID ###

In [70]:
aggregation_list = ['max', 'min', 'std', 'mean']
for aggregate in aggregation_list:
    train['target_Payment_Groupby_ID_Rate_' + aggregate] = train.groupby(['ID', 'RatePerUnit_cat'])['target_payment'].transform(aggregate)
    train['first_target_Payment_Groupby_ID_Rate_' + aggregate] = train.groupby(['ID', 'RatePerUnit_cat'])['first_target_payment'].transform(aggregate)

    test['target_Payment_Groupby_ID_Rate_' + aggregate] = test.groupby(['ID', 'RatePerUnit_cat'])['target_payment'].transform(aggregate)
    test['first_target_Payment_Groupby_ID_Rate_' + aggregate] = test.groupby(['ID', 'RatePerUnit_cat'])['first_target_payment'].transform(aggregate)

### DaysOnDeposit_week_list AND ID ###


In [71]:
aggregation_list = ['max', 'min', 'std', 'mean']
for aggregate in aggregation_list:
    train['target_Payment_Groupby_ID_DaysOnDeposit_week_' + aggregate] = train.groupby(['ID', 'DaysOnDeposit_week_list'])['target_payment'].transform(aggregate)
    train['first_target_Payment_Groupby_ID_DaysOnDeposit_week_' + aggregate] = train.groupby(['ID', 'DaysOnDeposit_week_list'])['first_target_payment'].transform(aggregate)

    test['target_Payment_Groupby_ID_DaysOnDeposit_week_' + aggregate] = test.groupby(['ID', 'DaysOnDeposit_week_list'])['target_payment'].transform(aggregate)
    test['first_target_Payment_Groupby_ID_DaysOnDeposit_week_' + aggregate] = test.groupby(['ID', 'DaysOnDeposit_week_list'])['first_target_payment'].transform(aggregate)

# Mean Encoding of Categorical Columns

In [72]:
# mean_encoding_combination = [
#     ['DatetimeLastPaymentDate','Sum_PaymentsHistory'],
#     ['Mean_PaymentsHistory', 'DaysOnDeposit_week_list'],
#     ['DatetimeLastPaymentDate', 'Sum_PaymentsHistory',  'month_six'],
#     ['DatetimeLastPaymentDate', 'HourofdayUpsellDate', 'Median_PaymentsHistory'],
#     ['RatePerUnit_cat', 'count_PaymentsHistory'],
#     ['RatePerUnit_cat', 'Sum_PaymentsHistory'],
#     ['RatePerUnit_cat', 'Sum_PaymentsHistory', 'count_PaymentsHistory'],
#     ['RatePerUnit_cat', 'DatetimeLastPaymentDate', 'count_PaymentsHistory'],
#     ['RatePerUnit_cat', 'DatetimeLastPaymentDate', 'Sum_PaymentsHistory']
# ]

# for col in mean_encoding_combination:
#     print(col, 'encoding')
#     colnm1 = '_'.join(col)+'_mean_enc'
#     colnm2 = '_'.join(col)+'_std_enc'
#     train[colnm1] = train.groupby(col)['target'].transform('mean')
#     train[colnm2] = train.groupby(col)['target'].transform('std')

# Mathematical Operations - Target Payment from First Target Payment

In [74]:
train['TargetPayment_Minus_FirstTargetPayment'] = train['target_payment'] - train['first_target_payment'] 
train['TargetPayment_Minus_FirstTargetPayment/2'] = train['TargetPayment_Minus_FirstTargetPayment']/2 
train['TargetPayment_Add_FirstTargetPayment'] = train['target_payment'] + train['first_target_payment'] 
train['TargetPayment_Add_FirstTargetPayment/2'] = train['TargetPayment_Add_FirstTargetPayment']/2
train['TargetPayment_Div_FirstTargetPayment'] = train['target_payment'] / train['first_target_payment'] 
train['FirstTargetPayment_Div_TargetPayment'] = train['first_target_payment'] / train['target_payment'] 

############################ TEST SET ######################################

test['TargetPayment_Minus_FirstTargetPayment'] = test['target_payment'] - test['first_target_payment'] 
test['TargetPayment_Minus_FirstTargetPayment/2'] = test['TargetPayment_Minus_FirstTargetPayment']/2 
test['TargetPayment_Add_FirstTargetPayment'] = test['target_payment'] + test['first_target_payment'] 
test['TargetPayment_Add_FirstTargetPayment/2'] = test['TargetPayment_Add_FirstTargetPayment']/2
test['TargetPayment_Div_FirstTargetPayment'] = test['target_payment'] / test['first_target_payment'] 
test['FirstTargetPayment_Div_TargetPayment'] = test['first_target_payment'] / test['target_payment'] 

In [75]:
train['MeanPayement_Add_DepositWeek'] = train['Mean_PaymentsHistory'] + train['first_shift_1_target_payment'] 
train['MeanPayement_Minus_DepositWeek'] = train['Mean_PaymentsHistory'] - train['first_shift_1_target_payment'] 

test['MeanPayement_Add_DepositWeek'] = test['Mean_PaymentsHistory'] + test['first_shift_1_target_payment'] 
test['MeanPayement_Minus_DepositWeek'] = test['Mean_PaymentsHistory'] - test['first_shift_1_target_payment'] 



# Dealing with the Date column

In [76]:
train[['target_date_month_Numeric', 'target_date_year']] = train.date_target_payment.str.split("-",expand=True)
test[['target_date_month_Numeric', 'target_date_year']] = test.date_target_payment.str.split("-",expand=True)

train[['first_target_date_month_Numeric', 'first_target_date_year']] = train.date_first_target_payment.str.split("-",expand=True)
test[['first_target_date_month_Numeric', 'first_target_date_year']] = test.date_first_target_payment.str.split("-",expand=True)

##################################### DELETE UNUESEFUL COLUMNS ###################################################
del train['target_date_year']
del test['target_date_year']
del train['first_target_date_year']
del test['first_target_date_year']

###################### Also use it as a categorical column  ##################
train['target_date_month_Category'] = train['target_date_month_Numeric']
train['first_target_date_month_Category'] = train['first_target_date_month_Numeric'] 

test['target_date_month_Category'] =  test['target_date_month_Numeric']
test['first_target_date_month_Category'] = test['first_target_date_month_Numeric']

################ CONVERT THE NUMERIC COLUMNS TO REAL NUMERIC COLUMNS #############
train['target_date_month_Numeric'] =  pd.to_numeric(train['target_date_month_Numeric'] )
train['first_target_date_month_Numeric'] =pd.to_numeric(train['first_target_date_month_Numeric']) 

test['target_date_month_Numeric'] = pd.to_numeric(test['target_date_month_Numeric']) 
test['first_target_date_month_Numeric'] = pd.to_numeric(test['first_target_date_month_Numeric']) 


del train['date_target_payment']
del train['date_first_target_payment']

del test['date_target_payment']
del test['date_first_target_payment']


# Joining Features Together

In [77]:
train['join_Occupation_and_Town'] = train['Occupation'] + "_" + train['Town']
test['join_Occupation_and_Town'] = test['Occupation'] + "_" + test['Town']


train['join_Occupation_and_Region'] = train['Occupation'] + "_" + train['Region']
test['join_Occupation_and_Region'] = test['Occupation'] + "_" + test['Region']


train['join_Town_and_Region'] = train['Town'] + "_" + train['Region']
test['join_Town_and_Region'] = test['Town'] + "_" + test['Region']


train['join_rateTypeEntity_and_RatePerUnit'] = train['rateTypeEntity'] + "_" + (train['RatePerUnit'].map(str))
test['join_rateTypeEntity_and_RatePerUnit'] = test['rateTypeEntity'] + "_" + (test['RatePerUnit'].map(str))
	

# Grouping

In [78]:
aggregation_list = ['max', 'min', 'std', 'mean', 'count']
for aggregate in aggregation_list:
    train["AveragePayment_Occupation_" + aggregate] = (train.groupby("Occupation")["target_payment"].transform(aggregate))
    train["AveragePayment_Town_" + aggregate] = (train.groupby("Town")["target_payment"].transform(aggregate))
    train["AveragePayment_Region_" + aggregate] = (train.groupby("Region")["target_payment"].transform(aggregate))
    train["AveragePayment_Age_" + aggregate] = (train.groupby("Age")["target_payment"].transform(aggregate))
    train["AveragePayment_rateTypeEntity_" + aggregate] = (train.groupby("rateTypeEntity")["target_payment"].transform(aggregate))
    train["AveragePayment_MainApplicantGender_" + aggregate] = (train.groupby("MainApplicantGender")["target_payment"].transform(aggregate))
    train["AveragePayment_AccessoryRate_" + aggregate] = (train.groupby("AccessoryRate")["target_payment"].transform(aggregate))

    train["first_AveragePayment_Occupation_" + aggregate] = (train.groupby("Occupation")["first_target_payment"].transform(aggregate))
    train["first_AveragePayment_Town_" + aggregate] = (train.groupby("Town")["first_target_payment"].transform(aggregate))
    train["first_AveragePayment_Region_" + aggregate] = (train.groupby("Region")["first_target_payment"].transform(aggregate))
    train["first_AveragePayment_Age_" + aggregate] = (train.groupby("Age")["first_target_payment"].transform(aggregate))
    train["first_AveragePayment_rateTypeEntity_" + aggregate] = (train.groupby("rateTypeEntity")["first_target_payment"].transform(aggregate))
    train["first_AveragePayment_MainApplicantGender_" + aggregate] = (train.groupby("MainApplicantGender")["first_target_payment"].transform(aggregate))
    train["first_AveragePayment_AccessoryRate_" + aggregate] = (train.groupby("AccessoryRate")["first_target_payment"].transform(aggregate))


    ##################################### TEST SET ##########################################################
    test["AveragePayment_Occupation_" + aggregate] = (test.groupby("Occupation")["target_payment"].transform(aggregate))
    test["AveragePayment_Town_" + aggregate] = (test.groupby("Town")["target_payment"].transform(aggregate))
    test["AveragePayment_Region_" + aggregate] = (test.groupby("Region")["target_payment"].transform(aggregate))
    test["AveragePayment_Age_" + aggregate] = (test.groupby("Age")["target_payment"].transform(aggregate))
    test["AveragePayment_rateTypeEntity_" + aggregate] = (test.groupby("rateTypeEntity")["target_payment"].transform(aggregate))
    test["AveragePayment_MainApplicantGender_" + aggregate] = (test.groupby("MainApplicantGender")["target_payment"].transform(aggregate))
    test["AveragePayment_AccessoryRate_" + aggregate] = (test.groupby("AccessoryRate")["target_payment"].transform(aggregate))

    test["first_AveragePayment_Occupation_" + aggregate] = (test.groupby("Occupation")["first_target_payment"].transform(aggregate))
    test["first_AveragePayment_Town_" + aggregate] = (test.groupby("Town")["first_target_payment"].transform(aggregate))
    test["first_AveragePayment_Region_" + aggregate] = (test.groupby("Region")["first_target_payment"].transform(aggregate))
    test["first_AveragePayment_Age_" + aggregate] = (test.groupby("Age")["first_target_payment"].transform(aggregate))
    test["first_AveragePayment_rateTypeEntity_" + aggregate] = (test.groupby("rateTypeEntity")["first_target_payment"].transform(aggregate))
    test["first_AveragePayment_MainApplicantGender_" + aggregate] = (test.groupby("MainApplicantGender")["first_target_payment"].transform(aggregate))
    test["first_AveragePayment_AccessoryRate_" + aggregate] = (test.groupby("AccessoryRate")["first_target_payment"].transform(aggregate))

In [79]:
aggregation_list = ['count']
for aggregate in aggregation_list:
    train["Occupation_" + aggregate] = ((train.groupby("Occupation")["Occupation"].transform(aggregate))/train.Occupation.count())
    train["Town_" + aggregate] = ((train.groupby("Town")["Town"].transform(aggregate))/train.Town.count())
    train["Region_" + aggregate] = ((train.groupby("Region")["Region"].transform(aggregate))/train.Region.count())
    train["Age_" + aggregate] = ((train.groupby("Age")["Age"].transform(aggregate))/train.Age.count())
    train["rateTypeEntity_" + aggregate] = ((train.groupby("rateTypeEntity")["rateTypeEntity"].transform(aggregate))/train.rateTypeEntity.count())
    train["MainApplicantGender_" + aggregate] = ((train.groupby("MainApplicantGender")["MainApplicantGender"].transform(aggregate))/train.MainApplicantGender.count())
    train["AccessoryRate_" + aggregate] = ((train.groupby("AccessoryRate")["AccessoryRate"].transform(aggregate))/train.AccessoryRate.count())

    ##################################### TEST SET ##########################################################
    test["Occupation_" + aggregate] = ((test.groupby("Occupation")["Occupation"].transform(aggregate))/train.Occupation.count())
    test["Town_" + aggregate] = ((test.groupby("Town")["Town"].transform(aggregate))/train.Town.count())
    test["Region_" + aggregate] = ((test.groupby("Region")["Region"].transform(aggregate))/train.Region.count())
    test["Age_" + aggregate] = ((test.groupby("Age")["Age"].transform(aggregate))/train.Age.count())
    test["rateTypeEntity_" + aggregate] = ((test.groupby("rateTypeEntity")["rateTypeEntity"].transform(aggregate))/train.rateTypeEntity.count())
    test["MainApplicantGender_" + aggregate] = ((test.groupby("rateTypeEntity")["rateTypeEntity"].transform(aggregate))/train.MainApplicantGender.count())
    test["AccessoryRate_" + aggregate] = ((test.groupby("AccessoryRate")["AccessoryRate"].transform(aggregate))/train.AccessoryRate.count())

# Plotting Trends and Seasonality

## For the target_payment - 2019 to 2020
## for first_target_payment - 2017 to 2018

# Dealing with the Defaulted Months of Payments engineered in the train and test dataset

In [80]:
train['default_times'] = train['defaulted_times'].str.extract('(\d+)').astype(int)
test['default_times'] = test['defaulted_times'].str.extract('(\d+)').astype(int)

del train['defaulted_times']
del test['defaulted_times']

################# CONVERT THE COLUMN TO NUMERIC AND ALSO CREATE CATEGORICAL DATA FROM IT ####################


train['default_times_Numeric'] = pd.to_numeric(train['default_times'])
test['default_times_Numeric'] = pd.to_numeric(test['default_times'])

del train['default_times']
del test['default_times']


train['default_times_Category'] = pd.to_numeric(train['default_times_Numeric'])
test['default_times_Category'] = pd.to_numeric(test['default_times_Numeric'])

# Categorizing into two Categories - Defaulted or Not_Defaulted

In [86]:
train['defaulted_Category'] = train.default_times_Numeric.apply(lambda x: "YES" if x > 0 else "NO")
train['defaulted_Numeric'] = train.default_times_Numeric.apply(lambda x: 1 if x > 0 else 0)

test['defaulted_Category'] = test.default_times_Numeric.apply(lambda x: "YES" if x > 0 else "NO")
test['defaulted_Numeric'] = test.default_times_Numeric.apply(lambda x: 1 if x > 0 else 0)

# Expanding Mean for Categorical Features

In [103]:
cat_cols = ['UpsellDate', 'rateTypeEntity', 'MainApplicantGender', 'Region', 'Town', 'Occupation', 'weekdayRegistrationDate', 'weekdayUpsellDate', 
'weekdayExpectedTermDate', 'weekdayFirstPaymentDate', 'weekdayLastPaymentDate', 'RatePerUnit_cat', 'month_List'] 

for col in cat_cols:
    cumsum = train.groupby(col)['target_payment'].cumsum() - train['target_payment']
    cumcnt = train.groupby(col).cumcount()
    train[col +'_expanding_mean_Target_Payment'] = cumsum/cumcnt

    cumsum = train.groupby(col)['first_target_payment'].cumsum() - train['first_target_payment']
    cumcnt = train.groupby(col).cumcount()
    train[col +'_expanding_mean_First_Target_Payment'] = cumsum/cumcnt

    ################################# TEST SET ##################################
    cumsum = test.groupby(col)['target_payment'].cumsum() - test['target_payment']
    cumcnt = test.groupby(col).cumcount()
    test[col +'_expanding_mean_Target_Payment'] = cumsum/cumcnt

    cumsum = test.groupby(col)['first_target_payment'].cumsum() - test['first_target_payment']
    cumcnt = test.groupby(col).cumcount()
    test[col +'_expanding_mean_First_Target_Payment'] = cumsum/cumcnt

In [107]:
############################## FILL NAN VALUES ######################
nan_cols = train.columns[train.isnull().any()].tolist()
for col in nan_cols:
    train = train.fillna(train[col].value_counts().index[0])
    test = test.fillna(test[col].value_counts().index[0])

In [120]:
print(train.shape)
print(test.shape)

(168042, 457)
(56016, 456)


# EXPORT DATASET

In [118]:
train.to_csv(r'C:\Solar Credit Repayment\Zipped_Solar\full_Engineered_train.csv')
test.to_csv(r'C:\Solar Credit Repayment\Zipped_Solar\full_Engineered_test.csv')

# Boruta Engineer Data

In [122]:
train_boruta = train.iloc[:84021, :]
train_boruta.to_csv(r'C:\Solar Credit Repayment\Zipped_Solar\boruta_full_Engineered_train.csv')

In [33]:
train_boruta = train.iloc[:84021, :]
train_boruta.to_csv(r'C:\Solar Credit Repayment\Zipped_Solar\boruta_full_Engineered_train.csv')
train_boruta= pd.read_csv(r'C:\Solar Credit Repayment\Zipped_Solar\boruta_full_Engineered_train.csv')

# Boruta Analysis

In [51]:
cols = train_.columns.tolist()
cat_cols=  [col for col in cols if train_[col].dtype == 'O' or train_[col].dtype == 'str' or 'Category' in col]
print(cat_cols)

# remove_cat_cols = ['mean_Conditon_Target_Payment', 'median_Conditon_Target_Payment', 'DatetimeUpsellDate', 'DatetimeRegistrationDate', 
# 'ExpectedTermDate', 'FirstPaymentDate', 'LastPaymentDate','DatetimeExpectedTermDate', 'DatetimeFirstPaymentDate', 'DatetimeLastPaymentDate']

for col in remove_cat_cols:
    cat_cols.remove(col)

loc_one = [train_.columns.get_loc(c) for c in cat_cols if c in train_]
loc_two = []
for value in loc_one:
    value_added = value + 1
    loc_two.append(value_added)
print(loc_two)
    

['ID X Month', 'TransactionDates', 'PaymentsHistory', 'ID', 'RegistrationDate', 'UpsellDate', 'PaymentMethod', 'rateTypeEntity', 'MainApplicantGender', 'Region', 'Town', 'Occupation', 'SupplierName', 'ExpectedTermDate', 'FirstPaymentDate', 'LastPaymentDate', 'DatetimeRegistrationDate', 'weekdayRegistrationDate', 'DatetimeUpsellDate', 'weekdayUpsellDate', 'DatetimeExpectedTermDate', 'weekdayExpectedTermDate', 'DatetimeFirstPaymentDate', 'weekdayFirstPaymentDate', 'DatetimeLastPaymentDate', 'weekdayLastPaymentDate', 'month_List', 'mean_Conditon_Target_Payment', 'median_Conditon_Target_Payment', 'target_date_month_Category', 'first_target_date_month_Category', 'join_Occupation_and_Town', 'join_Occupation_and_Region', 'join_Town_and_Region', 'join_rateTypeEntity_and_RatePerUnit', 'default_times_Category', 'defaulted_Category']
[2, 3, 4, 5, 33, 35, 37, 38, 41, 43, 44, 45, 46, 60, 82, 104, 126, 148, 185, 346, 347, 348, 349, 350, 351, 430, 431]


In [64]:
train_.iloc[:, 185]

0         YES
1          NO
2          NO
3          NO
4         YES
         ... 
168037    YES
168038    YES
168039    YES
168040    YES
168041    YES
Name: mean_Conditon_Target_Payment, Length: 168042, dtype: object

In [ ]:
'TransactionDates', 'PaymentsHistory'


# SAVE MODEL 

In [91]:
import pickle
filename = r'C:\Solar Credit Repayment\Zipped_Solar\finalized_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [97]:
# Create the model  
random.seed(3)

# del model
model = CatBoostRegressor(iterations=1000, 
                           loss_function='RMSE', 
                           eval_metric='RMSE',
                           verbose=200) 

model.fit(X_train, y_train, cat_features=cat_mean_col, eval_set=(X_val, y_val)) # Takes about 

Learning rate set to 0.114942
0:	learn: 896.3193296	test: 813.2046813	best: 813.2046813 (0)	total: 142ms	remaining: 2m 21s
200:	learn: 739.7147782	test: 707.0446091	best: 703.7429034 (94)	total: 28.4s	remaining: 1m 52s
400:	learn: 716.2413054	test: 711.9050084	best: 703.7429034 (94)	total: 1m 2s	remaining: 1m 33s
600:	learn: 692.3322334	test: 715.4574337	best: 703.7429034 (94)	total: 1m 39s	remaining: 1m 6s
800:	learn: 677.3858816	test: 719.8761195	best: 703.7429034 (94)	total: 2m 8s	remaining: 31.9s
999:	learn: 666.6994760	test: 724.0582782	best: 703.7429034 (94)	total: 2m 40s	remaining: 0us

bestTest = 703.7429034
bestIteration = 94

Shrink model to first 95 iterations.


In [100]:
i = 0
for col in main_cols:
   
    print(i, col)
    i = i + 1

0 Mean_PaymentsHistory
1 Median_PaymentsHistory
2 MostCommon_PaymentsHistory
3 variance_PaymentsHistory
4 stdev_PaymentsHistory
5 HighestPay_PaymentsHistory
6 LowestPay_PaymentsHistory
7 Sum_PaymentsHistory
8 count_PaymentsHistory
9 Deposit
10 AccessoryRate
11 PaymentMethod
12 rateTypeEntity
13 RatePerUnit
14 DaysOnDeposit
15 MainApplicantGender
16 Age
17 Region
18 Town
19 Occupation
20 SupplierName
21 Term
22 TotalContractValue
